<a href="https://colab.research.google.com/github/jsebastianquiroga/Semestre_3/blob/main/DL/Convolucion_2d.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from scipy import signal
import matplotlib.pyplot as plt
import numpy as np
from skimage import color
from skimage import io

from matplotlib.image import imread

In [ ]:
# Conexion a gdrive
from pydrive.auth import GoogleAuth
from google.colab import drive
from pydrive.drive import GoogleDrive
from google.colab import auth

from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

Booksummaries_share = '1xU2UNslewl5MOZbXSff6XEjA_odaX6rv' 
download = drive.CreateFile({'id': Booksummaries_share}) 
download.GetContentFile('booksummaries.txt')


df_book_summaries = pd.read_csv("booksummaries.txt", header=None, sep="\t", names=['Wiki_ID', 'Freebase_ID', 'Title', 'Author', 'Pub_date', 'Genres', 'Summary'])

df_book_summaries

In [ ]:
filename = os.path.join('imagen/', 'cuadrado7x7.jpg')
imagen  = imread(filename)
imagen2  = imread(filename)